In [1]:
from data_storage import create_connection
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from stockstats import StockDataFrame
import os
import ta
from pyti.chande_momentum_oscillator import chande_momentum_oscillator
from pyti.accumulation_distribution import accumulation_distribution
from pyti.average_true_range_percent import average_true_range_percent


In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
# function to load data into database
def load_data_into_database(path, db_connection, header_list):
    
    for file in os.listdir(path):
            
            if file[-3:] == 'txt':
                file_name = str(file).replace("-", "_")
                df = pd.read_csv(f'./{path}/{file}', names = header_list)
                
                df.to_sql(f'{file_name[:-4]}_complete_raw', con=db_connection, if_exists="replace", index=False)

In [4]:
headers = ["time", "open", "high", "low", "close", "volume"]
load_data_into_database("../database/", connection, headers)

In [5]:
#resamples the data to a faily format as described in chapter 3.3
def preprocessing_data_1day(db_connection):
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_complete_raw" in name and 'trades' not in name]
    print(filtered_table_names)
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        
        df_temp["time"] = pd.to_datetime(df_temp['time'])
        df_temp = df_temp.set_index('time')
        
        df_temp = df_temp.resample('1D').agg({"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum", })
        
        df_temp["volume"] = df_temp["volume"].fillna(0)
        df_temp["close"] = df_temp["close"].fillna(method="ffill")
        df_temp = df_temp.fillna(axis=1, method="backfill")
      
    
        df_temp.to_sql(f"{table[:-4]}_1day_preprocessed", db_connection, if_exists="replace")

In [6]:
preprocessing_data_1day(connection)

['ADA_1min_complete_raw', 'BCH_1min_complete_raw', 'BNT_1min_complete_raw', 'BSV_1min_complete_raw', 'BTC_1min_complete_raw', 'BTG_1min_complete_raw', 'DASH_1min_complete_raw', 'DOGE_1min_complete_raw', 'EOS_1min_complete_raw', 'ETC_1min_complete_raw', 'ETH_1min_complete_raw', 'FUN_1min_complete_raw', 'ICX_1min_complete_raw', 'KNC_1min_complete_raw', 'LINK_1min_complete_raw', 'LRC_1min_complete_raw', 'LTC_1min_complete_raw', 'MKR_1min_complete_raw', 'NEO_1min_complete_raw', 'OMG_1min_complete_raw', 'ONT_1min_complete_raw', 'QTUM_1min_complete_raw', 'REP_1min_complete_raw', 'SNT_1min_complete_raw', 'TRX_1min_complete_raw']


In [7]:
def momentum2(df):
    df["return"] = df["close"] / df["open"] - 1
    return df["return"]

# def momentum(df, lag):
#     return df.pct_change(periods=lag)

In [8]:
#function to create the target variables
def create_target_variable(df_target):
    
    df_target['return'] = momentum2(df_target)
        
    df_target["mean_return"] = df_target["return"].rolling(50).mean()
        
    df_target["std_deviation"] = df_target["return"].rolling(50).std()
        
    df_target["buy_indicator"] = 0
    df_target.loc[df_target["return"] > df_target["mean_return"] + 1 * df_target["std_deviation"], 'buy_indicator'] = 1
    df_target["buy_indicator"] = df_target["buy_indicator"].shift(-1)
    df_target["close_buy_indicator"] = df_target["buy_indicator"].shift(1)
        
        
    df_target["short_indicator"] = 0
    df_target.loc[df_target["return"] < df_target["mean_return"] - 1 * df_target["std_deviation"], 'short_indicator'] = -1
    df_target["short_indicator"] = df_target["short_indicator"].shift(-1)
    df_target["close_short_indicator"] = df_target["short_indicator"].shift(1).fillna(0)
    
    return df_target["buy_indicator"], df_target["short_indicator"], df_target["close_buy_indicator"], df_target["close_short_indicator"]
        

In [9]:
#creates features and target variable
def create_features(db_connection):
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_preprocessed" in name and 'trades' not in name and "_features" not in name]
    print(filtered_table_names)
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        
        df_ti = pd.DataFrame()
        df_ti["time"] = df_temp["time"]
        df_ti["open"] = df_temp["open"]
        df_ti["close"] = df_temp["close"]
        df_ti["high"] = df_temp["high"]
        df_ti["low"] = df_temp["low"]
        df_ti["volume"] = df_temp["volume"]
    
        stock = StockDataFrame.retype(df_ti)
        
        df_temp["sma5-20"] = stock.get("close_5_sma") - stock.get("close_20_sma")
        df_temp["sma8-15"] = stock.get("close_8_sma") - stock.get("close_15_sma")
        df_temp["sma20-50"] = stock.get("close_20_sma") - stock.get("close_50_sma")
        df_temp["ema5-20"] = stock.get("close_5_ema") - stock.get("close_20_ema")
        df_temp["ema8-15"] = stock.get("close_8_ema") - stock.get("close_15_ema")
        df_temp["ema20-50"] = stock.get("close_20_ema") - stock.get("close_50_ema")
        df_temp["macd"] = stock.get("macd")
        df_temp["ao14"] = ta.trend.AroonIndicator(df_temp["close"], window = 14, fillna=True).aroon_indicator() 
        df_temp["adx14"] = ta.trend.ADXIndicator(df_temp["high"], df_temp["low"], df_temp["close"], 14).adx()
        df_temp["wd14"] = stock.get("pdi_14") - stock.get("mdi_14")
        
        df_temp["ppo12-26"] = stock.get("ppo") #default is 14
        df_temp["rsi14"] = stock.get("rsi_14")
        df_temp["mfi14"] = stock.get("mfi_14")
        df_temp["tsi"] = ta.momentum.TSIIndicator(df_temp["close"], fillna=True).tsi()
        df_temp["so14"] = stock.get("kdjk_14")
        df_temp["cmo14"] = chande_momentum_oscillator(df_temp["close"], 14)
        df_temp["atrp14"] = average_true_range_percent(df_temp["close"], 14)
        
        df_temp["pvo12-26"] = ta.momentum.PercentageVolumeOscillator(df_temp["volume"], fillna=True).pvo()
        df_temp["adl"] = accumulation_distribution(df_temp["close"], df_temp["high"], df_temp["low"], df_temp["volume"])
        df_temp["obv"] = ta.volume.OnBalanceVolumeIndicator(df_temp["close"], df_temp["volume"]).on_balance_volume()
        df_temp["fi13"] = ta.volume.ForceIndexIndicator(df_temp["close"], df_temp["volume"], 13, fillna= True).force_index()
        df_temp["fi50"] = ta.volume.ForceIndexIndicator(df_temp["close"], df_temp["volume"], 50, fillna=True).force_index()
        df_temp["market_cap"] = df_temp["close"] * df_temp["volume"]
        
        df_temp = df_temp.fillna(method = "backfill")
        df_temp = df_temp.fillna(method ="ffill")
        
        df_temp["buy_indicator"], df_temp["short_indicator"], df_temp["close_buy_indicator"], df_temp["close_short_indicator"] = create_target_variable(df_temp)
        df_temp = df_temp.drop(['mean_return', 'std_deviation'], axis=1)
        
        df_temp.to_sql(f"{table}_1day_features", db_connection, if_exists="replace")
        
    

In [10]:
create_features(connection)

['ADA_1min_complete_1day_preprocessed', 'BCH_1min_complete_1day_preprocessed', 'BNT_1min_complete_1day_preprocessed', 'BSV_1min_complete_1day_preprocessed', 'BTC_1min_complete_1day_preprocessed', 'BTG_1min_complete_1day_preprocessed', 'DASH_1min_complete_1day_preprocessed', 'DOGE_1min_complete_1day_preprocessed', 'EOS_1min_complete_1day_preprocessed', 'ETC_1min_complete_1day_preprocessed', 'ETH_1min_complete_1day_preprocessed', 'FUN_1min_complete_1day_preprocessed', 'ICX_1min_complete_1day_preprocessed', 'KNC_1min_complete_1day_preprocessed', 'LINK_1min_complete_1day_preprocessed', 'LRC_1min_complete_1day_preprocessed', 'LTC_1min_complete_1day_preprocessed', 'MKR_1min_complete_1day_preprocessed', 'NEO_1min_complete_1day_preprocessed', 'OMG_1min_complete_1day_preprocessed', 'ONT_1min_complete_1day_preprocessed', 'QTUM_1min_complete_1day_preprocessed', 'REP_1min_complete_1day_preprocessed', 'SNT_1min_complete_1day_preprocessed', 'TRX_1min_complete_1day_preprocessed']


c:\Users\janfa\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\janfa\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
c:\Users\janfa\anaconda3\lib\site-packages\pyti\accumulation_distribution.py:21: RuntimeWarning: invalid value encountered in double_scalars
  (((close_data[idx] - low_data[idx]) -
c:\Users\janfa\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\janfa\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
c:\Users\janfa\anaconda3\lib\site-packages\pyti\accumulation_distribution.py:21: RuntimeWarning: invalid value encountered in double_scalars
  (((close_data[idx] -

In [11]:
#a market cap filter which filters cryptocurrencies which a have not reached a daily market cap of 10.000.000 once
#the trading data of a crypto was used once the market cap is reached
def market_cap_filter(db_connection):
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_features" in name and 'trades' not in name and 'equity_curve' not in name and '_pooling' not in name]
    print(filtered_table_names)
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"SELECT * FROM {table}", db_connection)
        #filter set to 10.000.000
        filter_index = df_temp.index[df_temp['market_cap'] >= 10000000].tolist()
        
    
        if len(filter_index) == 0:
            print(table)
            df_temp = df_temp[0:0]
            df_temp.to_sql(f"{table}", db_connection, if_exists="replace")
        else:
            first_break_point = filter_index[0]
            df_stage2 = df_temp.iloc[first_break_point:].copy()
            
            df_stage2.to_sql(f"{table}", db_connection, if_exists="replace")
           
            

In [12]:
market_cap_filter(connection)

['ADA_1min_complete_1day_preprocessed_1day_features', 'BCH_1min_complete_1day_preprocessed_1day_features', 'BNT_1min_complete_1day_preprocessed_1day_features', 'BSV_1min_complete_1day_preprocessed_1day_features', 'BTC_1min_complete_1day_preprocessed_1day_features', 'BTG_1min_complete_1day_preprocessed_1day_features', 'DASH_1min_complete_1day_preprocessed_1day_features', 'DOGE_1min_complete_1day_preprocessed_1day_features', 'EOS_1min_complete_1day_preprocessed_1day_features', 'ETC_1min_complete_1day_preprocessed_1day_features', 'ETH_1min_complete_1day_preprocessed_1day_features', 'FUN_1min_complete_1day_preprocessed_1day_features', 'ICX_1min_complete_1day_preprocessed_1day_features', 'KNC_1min_complete_1day_preprocessed_1day_features', 'LINK_1min_complete_1day_preprocessed_1day_features', 'LRC_1min_complete_1day_preprocessed_1day_features', 'LTC_1min_complete_1day_preprocessed_1day_features', 'MKR_1min_complete_1day_preprocessed_1day_features', 'NEO_1min_complete_1day_preprocessed_1day_

In [13]:
#creates the pooling dataset of all crypto datasets
def create_pooling_dataset(db_connection):
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_features" in name and 'trades' not in name and 'equity_curve' not in name and '_pooling' not in name]
    print(filtered_table_names)
    union_all_sql_list = []
    
    for table in filtered_table_names[:-1]:
        
        union_new_table = f"SELECT * FROM {table } where time < '2021-04-01 00:00:00' UNION ALL"
        union_all_sql_list.append(union_new_table)
    
    union_all_sql_list.append(f"SELECT * from {filtered_table_names[-1]} where time < '2021-04-01 00:00:00'")

    union_all_sql = ' '.join(union_all_sql_list)
    
    df = pd.read_sql_query(union_all_sql, db_connection)
    df = df.drop(["level_0"], axis=1)
    print(len(df)) #11973 #7720

    df.to_sql("cryptocurrency_pooling_dataset", db_connection, if_exists="replace")

In [14]:
create_pooling_dataset(connection)

['ADA_1min_complete_1day_preprocessed_1day_features', 'BCH_1min_complete_1day_preprocessed_1day_features', 'BNT_1min_complete_1day_preprocessed_1day_features', 'BSV_1min_complete_1day_preprocessed_1day_features', 'BTC_1min_complete_1day_preprocessed_1day_features', 'BTG_1min_complete_1day_preprocessed_1day_features', 'DASH_1min_complete_1day_preprocessed_1day_features', 'DOGE_1min_complete_1day_preprocessed_1day_features', 'EOS_1min_complete_1day_preprocessed_1day_features', 'ETC_1min_complete_1day_preprocessed_1day_features', 'ETH_1min_complete_1day_preprocessed_1day_features', 'FUN_1min_complete_1day_preprocessed_1day_features', 'ICX_1min_complete_1day_preprocessed_1day_features', 'KNC_1min_complete_1day_preprocessed_1day_features', 'LINK_1min_complete_1day_preprocessed_1day_features', 'LRC_1min_complete_1day_preprocessed_1day_features', 'LTC_1min_complete_1day_preprocessed_1day_features', 'MKR_1min_complete_1day_preprocessed_1day_features', 'NEO_1min_complete_1day_preprocessed_1day_